In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import time
import os
import nltk as nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import plot_confusion_matrix,classification_report
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emansell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/emansell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
rawdf = pd.read_csv('train.csv')
df = rawdf.dropna()
df

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...
...,...,...,...,...,...
290178,bobby womack,i wish he didn t trust me so much,R&B,en,I'm the best friend he's got I'd give him the ...
290179,bad boys blue,i totally miss you,Pop,en,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,celine dion,sorry for love,Pop,en,Forgive me for the things That I never said to...
290181,dan bern,cure for aids,Indie,en,The day they found a cure for AIDS The day the...


In [4]:
#Dropping genres, non-English lyrics
excessgenres = ['Folk', 'Indie', 'Other']

df = df[df.Genre.isin(excessgenres) == False]
df = df[df.Language == 'en']

In [5]:
#Taking random 900 songs from each genre
genres = ['Rock', 'Pop', 'Hip-Hop', 'Metal', 'Country', 'Jazz', 'Electronic', 'R&B']
rand = pd.DataFrame()
for genre in genres:
    newdf = df[df.Genre == genre]
    sample = newdf.sample(n = 900)
    rand = rand.append(sample)
rand   

/var/folders/dg/sxm0105x28g_bgwtjzwr0zc40000gn/T/ipykernel_64209/1614685821.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rand = rand.append(sample)
/var/folders/dg/sxm0105x28g_bgwtjzwr0zc40000gn/T/ipykernel_64209/1614685821.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rand = rand.append(sample)
/var/folders/dg/sxm0105x28g_bgwtjzwr0zc40000gn/T/ipykernel_64209/1614685821.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rand = rand.append(sample)
/var/folders/dg/sxm0105x28g_bgwtjzwr0zc40000gn/T/ipykernel_64209/1614685821.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rand = rand.append(sample)
/var/folders/dg/sxm0105x

,Artist,Song,Genre,Language,Lyrics
267372,"berry, chuck",every day i have the blues,Rock,en,every day\nevery day i have the blues\nevery d...
176056,april wine,all over town,Rock,en,"Someone said you was, runnin' 'round \nLike yo..."
49158,the rolling stones,love is strong,Rock,en,Love is strong\nAnd you're so sweet\nYou make ...
156827,madness,time,Rock,en,When I consider how my life is spent\nThey cry...
240798,the monkees,dream world,Rock,en,Walking around with your head in the clouds\nY...
...,...,...,...,...,...
252175,ruben studdard,can i get your attention,R&B,en,Kelly give me the ball girl\n\nYou are my girl...
163146,amel larrieux,congo,R&B,en,"Congo, get to, get to Congo\nGet to, get to Co..."
257250,jacob banks,sink or swim,R&B,en,head-first in the deep\ni must sink or swim\nl...
270091,az yet,every little bit of my heart,R&B,en,\ngirl\ni always think about it 24/7\ncan't ge...


In [6]:
#Create lemmatized corpus and clean lyrics
nlp = spacy.load('en_core_web_sm')
lemmatizedcorpus = []
def clean(text):
    # Make lower
    text = text.lower()
    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")+ ['solo', 'im', 'youre']
    text_filtered = [word for word in text if not word in useless_words]
    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]
    #Lemmatize and add to corpus
    text_joined = nlp(' '.join(text_filtered))
    text_stemmed = [y.lemma_ for y in text_joined]
    lemmatizedcorpus.append(str(nlp(' '.join(text_stemmed))))

    return ' '.join(text_stemmed)

rand['Cleaned Lyrics'] = rand.apply(lambda row: clean(row.Lyrics), axis = 1)


In [7]:
#Creating TFIDF matrix
tfidf = TfidfVectorizer(stop_words='english', lowercase=False)    
X = tfidf.fit_transform(lemmatizedcorpus)

tfidf_tokens = tfidf.get_feature_names_out()


matrix = pd.DataFrame(
    data=X.toarray(), 
    index=rand.iterrows(), 
    columns=tfidf_tokens
)

In [8]:
matrix

,aa,aaa,aaaaaaaaaahhhhhhhhhhhhh,aaaaahaaaaaheeey,aaaaass,aaaah,aaaarmagedoooon,aaaay,aaaaye,aaah,...,zuzu,zwei,zweier,zy,zydeco,âº,ã¼ber,ïs,órale,ømnis
"(267372, [berry, chuck, every day i have the blues, Rock, en, every day\nevery day i have the blues\nevery day\nevery day i have the blues\nwhen you see me worried\nbaby\nbecause it's you i hate to lose\nnobody loves me\nnobody seems to care\nnobody loves me\nnobody seems to care\nspeaking of bad luck and trouble will you know i've had my share\ni'm gonna pack my suit-case\nmove on down the line\ni'm gonna pack my suit-case\nmoving down the line\nwell\nthere ain't nobody worryin' and there ain't nobody cryin'\nseems to me every day\nevery day i have the blues\nevery day\nevery day i have the blues., every day every day blue every day every day blue see worried baby hate lose nobody love nobody seem care nobody love nobody seem care speak bad luck trouble know I ve share go to pack suitcase move line go to pack suitcase move line well be not nobody worryin be not nobody cryin seem every day every day blue every day every day blue])",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(176056, [april wine, all over town, Rock, en, Someone said you was, runnin' 'round \nLike you owned the world \nIt's no secret, you're the talk of the town \nYou know, the best of the girls \n\nThat's right, and now you \nWalked right out of my heart \nYou know it's all over town \nI said you walked walked, right right, out of my heart \nYou know it's all over town \nNow it's all over \n\nYou're so neat, you're so cute, you're so fine \nYou know that everybody wants some action \nThey see you comin' and they act like it's cool \nBecause they know that something's happenin' \n\nThat's right, because you \nWalked right out of my heart \nAnd now it's all over town \nI said you walked walked, right right, out of my heart \nYou know the word's goin' 'round \nNow it's all over, now it's all over \n\nI don't care (he don't care) \nI don't care \n\nI don't care care what they say about us anymore \nI don't care what they want to do \nBaby all my heart is for you \nNo matter what happens \n\nThere's no reason why we still can't be friends \nYou know we all need someone we can talk to \nNo matter what they say, we both know I'm right \nAfter all we've been through \n\nAnd now you walked right out of my heart \nYou know it's all over town \nI said you walked (walked), right (right), out of my heart \nYou know the word's goin' 'round \n\nNow it's all over \nI said you walked, right out of my heart \nI said you walked (walked), right (right), out of my heart \nYou walked right out of my heart \nAnd now it's all over town, someone say runnin round like own world secret talk town know good girl that s right walk right heart know town say walk walk right right heart know town neat cute fine know everybody want action see comin act like cool know something happenin that s right walk right heart town say walk walk right right heart know word goin round do not care do not care do not care do not care care say we anymore do not care want baby heart matter happen there s reason still can not friend know need someone talk matter say know right we ve walk right heart know town say walk walk right right heart know word goin round say walk right heart say walked walk right right heart walk right heart town])",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(49158, [the rolling stones, love is strong, Rock, en, Love is strong\nAnd you're so sweet\nYou make me hard\nYou make me weak\nLove is strong\nAnd you're so sweet\nAnd some day, babe\nWe got to meet\nA glimpse of you\nWas all it took\nA stranger's glance\nIt got me hooked\nAnd I followed you\nAcross the stars\nI looked for you\nIn seedy bars\nWhat are you scared of, baby\nIt's more than just a dream\nI need some time\nWe make a beautiful team\nA beautiful team\nLove is strong\nAnd you're so sweet\nAn

In [16]:
#Processing for models
matrix['target'] = LabelEncoder().fit_transform(rand["Genre"])
target = matrix['target']

X_train,X_test,y_train,y_test = train_test_split(X, target, test_size=0.2, stratify = target)

GradientBoostingClassifier(n_estimators=500, random_state=123)

In [34]:
# Gradient Boosting Classifier

gbmodel = GradientBoostingClassifier(n_estimators=500, random_state=123)
gbmodel.fit(X_train, y_train)
print("Gradient Boosting accuracy score {:.2f} %\n".format(gbmodel.score(X_test,y_test)*100))

Gradient Boosting accuracy score 39.72 %



In [35]:
# Random Forest Classifier

rfmodel = RandomForestClassifier()
rfmodel.fit(X_train, y_train)
print("Random forest accuracy score {:.2f} %\n".format(rfmodel.score(X_test,y_test)*100))

Random forest accuracy score 41.39 %



In [40]:
# Naive Bayes Classifier

nbmodel = MultinomialNB()
nbmodel.fit(X_train.toarray(), y_train)
print("Naive Bayes accuracy score {:.2f} %\n".format(nbmodel.score(X_test.toarray(),y_test)*100))

Naive Bayes accuracy score 40.83 %



In [42]:
# Grid search to improve model performance (Gradient Boost)

parameters = {
    "learning_rate": [0.01, 0.025, 0.05, 0.1, 0.2],
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "n_estimators":[10]
    }

gcv = GridSearchCV(gbmodel,parameters)
gcv.fit(X_train, y_train.values.ravel())
print(gcv.best_params_)

{'learning_rate': 0.01, 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 10}


In [43]:
# Grid search to improve model performance (Random Forest)

parameters = {
    'n_estimators': [5,50,100],
    'max_depth': [2,10,20,None]
}

rcv = GridSearchCV(rfmodel,parameters)
rcv.fit(X_train, y_train.values.ravel())
print(rcv.best_params_)

{'max_depth': 20, 'n_estimators': 100}


In [45]:
# Grid search to improve model performance (Naive Bayes)

parameters ={'alpha': [0.00001, 0.001, 0.1, 1, 10, 100]}

ncv = GridSearchCV(nbmodel,parameters)
ncv.fit(X_train, y_train.values.ravel())
print(ncv.best_params_)

{'alpha': 0.1}


In [51]:
# Optimized Gradient Boosting Classifier

ogbmodel = GradientBoostingClassifier(learning_rate = 0.01, max_depth= 8, max_features= 'sqrt', n_estimators = 10)
ogbmodel.fit(X_train, y_train)
print("Gradient Boosting accuracy score {:.2f} %\n".format(ogbmodel.score(X_test,y_test)*100))

Gradient Boosting accuracy score 35.62 %



In [52]:
# Optimized Random Forest Classifier

orfmodel = RandomForestClassifier(max_depth= 20, n_estimators= 100)
orfmodel.fit(X_train, y_train)
print("Random forest accuracy score {:.2f} %\n".format(orfmodel.score(X_test,y_test)*100))

Random forest accuracy score 40.42 %



In [53]:
# Optimized Naive Bayes Classifier

onbmodel = MultinomialNB(alpha=0.1)
onbmodel.fit(X_train.toarray(), y_train)
print("Naive Bayes accuracy score {:.2f} %\n".format(onbmodel.score(X_test.toarray(),y_test)*100))

Naive Bayes accuracy score 41.88 %



In [9]:
#Graph results
import matplotlib.pyplot as plt

graph = pd.DataFrame(
    {'Name': ['Multinomial NB', 'Grid Search Multinomial NB', 'Gradient Booster', 
              'Grid Search Gradient Booster', 'Random Forest', 'Grid Search Random Forest'], 
     'Accuracy': [nbmodel.score(X_test.toarray(),y_test), onbmodel.score(X_test.toarray(),y_test), gbmodel.score(X_test,y_test), ogbmodel.score(X_test,y_test), rfmodel.score(X_test,y_test), orfmodel.score(X_test,y_test)]}
)

plt.bar("Name", "Accuracy", data = graph, color=["blue", "green","blue", "green","blue", "green"])
plt.xlabel("Model")
plt.ylabel("Accuracy")
plt.xticks(rotation = 90)
plt.axhline(y= 0.125, linewidth=1, color="k")
plt.title("Top 3 models final performance")
plt.show()

NameError: name 'nbmodel' is not defined